In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib2 import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"] 

In [5]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df.select_dtypes(include=object)


,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,RENT,Source Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
1,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
2,MORTGAGE,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
3,RENT,Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
4,MORTGAGE,Not Verified,Mar-2019,low_risk,n,w,May-2019,Individual,N,N
...,...,...,...,...,...,...,...,...,...,...
68812,RENT,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68813,RENT,Not Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68814,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
68815,MORTGAGE,Verified,Jan-2019,low_risk,n,f,May-2019,Individual,N,N


In [7]:
loans_df = pd.get_dummies(df, columns=["home_ownership", "verification_status", "pymnt_plan", "initial_list_status", "next_pymnt_d", "application_type","hardship_flag", "debt_settlement_flag"])

loans_df

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,0.0,8.0,...,0,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,0.0,17.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,0.0,8.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,1.0,10.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,1.0,14.0,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,Jan-2019,low_risk,9.60,0.0,0.0,9.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,Jan-2019,low_risk,29.07,0.0,0.0,8.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,Jan-2019,low_risk,14.86,0.0,0.0,5.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,Jan-2019,low_risk,9.96,0.0,1.0,21.0,...,1,1,1,0,0,1,1,0,1,1


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

loans_encoding = loans_df.copy()
loans_encoding['home_ownership'] = le.fit_transform(df['home_ownership'] )
loans_encoding['verification_status'] = le.fit_transform(df['verification_status'])
loans_encoding['pymnt_plan'] = le.fit_transform(df['pymnt_plan'])
loans_encoding['initial_list_status'] = le.fit_transform(df['initial_list_status'])
loans_encoding['next_pymnt_d'] = le.fit_transform(df['next_pymnt_d'])
loans_encoding['application_type'] = le.fit_transform(df['application_type'])
loans_encoding['hardship_flag'] = le.fit_transform(df['hardship_flag'])
loans_encoding['debt_settlement_flag'] = le.fit_transform(df['debt_settlement_flag'])

loans_encoding.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,hardship_flag_N,debt_settlement_flag_N,home_ownership,verification_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,0.0,8.0,...,1,1,3,1,0,1,1,0,0,0
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,0.0,17.0,...,1,1,1,2,0,1,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,0.0,8.0,...,1,1,1,2,0,1,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,1.0,10.0,...,1,1,3,2,0,1,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,1.0,14.0,...,1,1,1,0,0,1,1,0,0,0


In [9]:
loans_encoding.columns.tolist()

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'issue_d',
 'loan_status',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_

In [10]:
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
}

In [11]:
loans_encoding["issue_d"] = loans_encoding["issue_d"].apply(lambda x: months_num[x])
loans_encoding.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,hardship_flag_N,debt_settlement_flag_N,home_ownership,verification_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,66000.0,3,low_risk,27.24,0.0,0.0,8.0,...,1,1,3,1,0,1,1,0,0,0
1,25000.0,0.2000,929.09,105000.0,3,low_risk,20.23,0.0,0.0,17.0,...,1,1,1,2,0,1,1,0,0,0
2,20000.0,0.2000,529.88,56000.0,3,low_risk,24.26,0.0,0.0,8.0,...,1,1,1,2,0,1,1,0,0,0
3,10000.0,0.1640,353.55,92000.0,3,low_risk,31.44,0.0,1.0,10.0,...,1,1,3,2,0,1,1,0,0,0
4,22000.0,0.1474,520.39,52000.0,3,low_risk,18.76,0.0,1.0,14.0,...,1,1,1,0,0,1,1,0,0,0


In [12]:
loans_encoded = loans_encoding

In [13]:
loans_encoded.columns.tolist()

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'issue_d',
 'loan_status',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_

# Split the Data into Training and Testing

In [14]:
# Create our target
y = loans_encoding['loan_status']

# Create our features

X = loans_encoding

X = loans_encoding.drop("loan_status", axis=1)


In [15]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,issue_d,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,hardship_flag_N,debt_settlement_flag_N,home_ownership,verification_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,1.726172,21.778153,0.217766,0.497697,12.587340,0.126030,...,1.0,1.0,1.812779,0.669994,0.0,0.876121,0.616839,0.139660,0.0,0.0
std,10277.348590,0.048130,288.062432,1.155800e+05,0.743862,20.199244,0.718367,0.758122,6.022869,0.336797,...,0.0,0.0,0.941313,0.719105,0.0,0.329446,0.486161,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,1.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,1.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,1.000000,13.890000,0.000000,0.000000,8.000000,0.000000,...,1.0,1.0,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,2.000000,19.760000,0.000000,0.000000,11.000000,0.000000,...,1.0,1.0,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,2.000000,26.660000,0.000000,1.000000,16.000000,0.000000,...,1.0,1.0,3.000000,1.000000,0.0,1.000000,1.000000,0.000000,0.0,0.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,3.000000,999.000000,18.000000,5.000000,72.000000,4.000000,...,1.0,1.0,3.000000,2.000000,0.0,1.000000,1.000000,1.000000,0.0,0.0


In [16]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [18]:
# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=500, random_state=24)

In [20]:
rf_model = rf_model.fit(X_train_scaled, y_train)

In [21]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)





In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [23]:
# Calculated the balanced accuracy score

acc_score = accuracy_score(y_test, predictions)

acc_score

0.9963382737576286

In [24]:
# Display the confusion matrix


cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,34,52
Actual 1,11,17108


In [25]:
# Print the imbalanced classification report

print("Classification Report")
print(classification_report(y_test, predictions))

Classification Report
              precision    recall  f1-score   support

   high_risk       0.76      0.40      0.52        86
    low_risk       1.00      1.00      1.00     17119

    accuracy                           1.00     17205
   macro avg       0.88      0.70      0.76     17205
weighted avg       1.00      1.00      1.00     17205



In [26]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.01560107, 0.01185545, 0.02095587, 0.01282863, 0.01366932,
       0.01637841, 0.00340729, 0.00525012, 0.00859629, 0.00268026,
       0.01475115, 0.01038617, 0.02065531, 0.02152652, 0.06567128,
       0.0666363 , 0.06451477, 0.06749243, 0.01259084, 0.        ,
       0.        , 0.07098986, 0.00125087, 0.        , 0.        ,
       0.00483896, 0.0126835 , 0.00544811, 0.00608928, 0.00420863,
       0.00650575, 0.01173894, 0.01445507, 0.01152032, 0.00436701,
       0.00637398, 0.0139524 , 0.01056081, 0.01252849, 0.00658816,
       0.00802565, 0.00853618, 0.00812493, 0.01326875, 0.01376223,
       0.01288854, 0.00025846, 0.        , 0.01558559, 0.01847387,
       0.00957683, 0.00861406, 0.00668796, 0.01049582, 0.01017528,
       0.00384692, 0.00701451, 0.00792357, 0.00634985, 0.00810644,
       0.01031156, 0.0081177 , 0.00886979, 0.00854167, 0.00884984,
       0.        , 0.        , 0.00183686, 0.00603912, 0.00903152,
       0.00649658, 0.00254919, 0.        , 0.0140164 , 0.01448

In [27]:
# List the features sorted in descending order by feature importance

sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07098985587948511, 'last_pymnt_amnt'),
 (0.06749242801306918, 'total_rec_int'),
 (0.06663630148571599, 'total_pymnt_inv'),
 (0.06567128442636407, 'total_pymnt'),
 (0.06451476835693261, 'total_rec_prncp'),
 (0.02152652017575015, 'out_prncp_inv'),
 (0.02095586668358335, 'installment'),
 (0.020655314160927533, 'out_prncp'),
 (0.018473865638369575, 'mo_sin_old_rev_tl_op'),
 (0.016378408431569073, 'dti'),
 (0.015601067534957476, 'loan_amnt'),
 (0.015585588650727757, 'mo_sin_old_il_acct'),
 (0.014751151432119596, 'revol_bal'),
 (0.014486645317983746, 'total_bal_ex_mort'),
 (0.014455073949804578, 'total_bal_il'),
 (0.014016400359557287, 'tot_hi_cred_lim'),
 (0.013990826738892579, 'total_il_high_credit_limit'),
 (0.013952401474100899, 'max_bal_bc'),
 (0.013762226232393262, 'bc_open_to_buy'),
 (0.013669317808131106, 'issue_d'),
 (0.013268746542829018, 'avg_cur_bal'),
 (0.012888538100833772, 'bc_util'),
 (0.012828627576263008, 'annual_inc'),
 (0.012683504457586986, 'tot_cur_bal'),
 (0.012590

### Easy Ensemble AdaBoost Classifier

In [32]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# With the code below, base_estimator parameter is set by deafult=AdaBoostClassifier()
easy_e = EasyEnsembleClassifier(n_estimators=100, random_state=24)
easy_e.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=24)

In [33]:
# Calculated the balanced accuracy score
y_pred = easy_e.predict(X_test_scaled)
easy_e_acc_score = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy Score (Easy Ensemble Classifier): {easy_e_acc_score}")

Balanced Accuracy Score (Easy Ensemble Classifier): 0.920394787785094


In [34]:
# Display the confusion matrix

# Calculating the confusion matrix
easy_e_cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
easy_cm_df = pd.DataFrame(
    easy_e_cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
easy_cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,77,9
Actual low_risk,934,16185


In [35]:
# Print the imbalanced classification report
easy_ibalance_report = classification_report_imbalanced(y_test, y_pred)
print(easy_ibalance_report)

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.90      0.95      0.14      0.92      0.84        86
   low_risk       1.00      0.95      0.90      0.97      0.92      0.85     17119

avg / total       0.99      0.95      0.90      0.97      0.92      0.85     17205

